In [ ]:
import serial_test

s = serial_test.FoobarTest("COM3")

# s.setLinkLength(0, 130.0, 120.0)
# s.setLinkLength(1, 130.0, 120.0)
# s.setServo_X_Y_Position(0, -140.0, 0.0)
# s.setServo_X_Y_Position(1, 140.0, 0.0)
# s.setClickTime(1000)
# s.setClickPosition(20, 43)
# s.saveCfg()


In [2]:
import math

L1 = 130
L2 = 120

SL_X = -140
SR_X = 140

SL_Y = 0
SR_Y = 0


def __nm(x, y, sx, sy, l1, l2):
    x -= sx
    y -= sy

    d2 = x*x+y*y
    d = d2**0.5

    cos_theta1 = (d2 + l1*l1 - l2*l2) / (2*d*l1)
    theta1 = math.acos(cos_theta1)
    theta_aux = math.atan2(y, x)
    # print('theta aux = ', theta_aux)
    if abs(theta_aux) < (math.pi/2):
        return theta1 + math.pi/2 + theta_aux
    while theta_aux < 0:
        theta_aux += (math.pi * 2)
    return theta_aux - (theta1 + math.pi/2)


def move_py_fuckArm(x, y):
    left_angle = __nm(x, y, SL_X, SL_Y, L1, L2)
    right_angle = __nm(x, y, SR_X, SR_Y, L1, L2)

    left_angle *= (100/math.pi)
    right_angle *= (100/math.pi)

    # print(f'left :  {left_angle*1.8}, ({int(left_angle)})')
    # print(f'right :  {right_angle*1.8}, ({int(right_angle)})')

    return [left_angle, right_angle]


# X_MID = 15
X_MID = 13
Y_MID = -40
GAP = 20

POWER_BUTTON = [X_MID+GAP-5+79, Y_MID+3]

points = [
    POWER_BUTTON,
    [X_MID+1, Y_MID + GAP],
    [X_MID - GAP+10, Y_MID],
    [X_MID+13, Y_MID - GAP+5],
    [X_MID + GAP+4, Y_MID+3]
]

# print(move_py_fuckArm(points[0][0],points[0][1]))

In [ ]:
px,py = points[0]
ax,ay = move_py_fuckArm(px,py)
print(ax,ay)

In [12]:
import serial_test
import time

def moveSlow2(x,y):
    ax,ay = move_py_fuckArm(x,y)
    serial_test.move_slow(ax,ay)
    time.sleep(2)

px,py = points[0]
ax,ay = move_py_fuckArm(px,py)

if not serial_test.open("COM3"):
    print('err')
    exit(1)
print('success')
serial_test.set_servo_position(0,int(ax))
serial_test.set_servo_position(1,int(ay))
# serial_test.setClickTime(200)
time.sleep(1)
px,py = points[1]

for px,py in points[1:]:
    moveSlow2(px,py)
for _ in range(2):
    for px,py in points:
        moveSlow2(px,py)
    # serial_test.click()
    # time.sleep(1.5)
    # ax,ay = move_py_fuckArm(px,py)
    # serial_test.move_slow(ax,ay)
px,py = points[0]
moveSlow2(px,py)
serial_test.close()

success
